# Wind Climatology Maps

In [1]:
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import xarray as xr
import cmocean
from salishsea_tools import viz_tools
from salishsea_tools import geo_tools
from salishsea_tools import grid_tools
from salishsea_tools import utilities
%matplotlib inline
speed_cmap = cmocean.cm.speed

In [2]:
GEM = xr.open_dataset("https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSaAtmosphereGridV1")  # HDPRC latlon
nemo_mask =xr.open_dataset("https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSn2DMeshMaskV17-02") # NEMO land mask
nemo_grid = xr.open_dataset("https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSnBathymetryV17-02") # NEMO lat lon
winddata = xr.open_dataset("https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSaSurfaceAtmosphereFieldsV1") # Contains wind data

In [3]:
wind_u = winddata.u_wind.loc[winddata['time.year'].isin([2015, 2016, 2016, 2017, 2018])]
wind_v = winddata.v_wind.loc[winddata['time.year'].isin([2015, 2016, 2016, 2017, 2018])]

In [4]:
def build_GEM_mask(grid_GEM, grid_NEMO, mask_NEMO):


    # Preallocate
    ngrid_GEM = grid_GEM['gridX'].shape[0] * grid_GEM['gridY'].shape[0]
    mask_GEM = np.zeros(ngrid_GEM, dtype=int)

    # Evaluate each point on GEM grid
    bar = utilities.statusbar('Building GEM mask', width=90, maxval=ngrid_GEM)
    for index, coords in enumerate(bar(zip(
            grid_GEM['longitude'].values.reshape(ngrid_GEM) - 360,
            grid_GEM['latitude'].values.reshape(ngrid_GEM),
    ))):

        j, i = geo_tools.find_closest_model_point(
            coords[0], coords[1],
            grid_NEMO['longitude'], grid_NEMO['latitude'],
        )
        if j is np.nan or i is np.nan:
            mask_GEM[index] = 0
        else:
            mask_GEM[index] = mask_NEMO[j, i].values

    # Reshape
    mask_GEM = mask_GEM.reshape(grid_GEM['longitude'].shape)

    return mask_GEM

In [5]:
land_mask = build_GEM_mask(GEM, nemo_grid, nemo_mask.tmaskutil.isel(time=0))

Building GEM mask 100% (68096 of 68096) |##################################|Time:  0:07:07


In [6]:
def mask_data(data, land_mask = land_mask):
    masked_data = ma.masked_array(data, np.logical_not(land_mask))  # mask the data
    remove_zero_rows = masked_data[~(masked_data==np.nan).all(1)]   # remove rows with only zero entries
    transpose_rows = np.transpose(remove_zero_rows)                 # transpose the resulting array
    remove_zero_cols = np.transpose(transpose_rows[~(transpose_rows==np.nan).all(1)])  # remove the column with only zero entries
    return remove_zero_cols

## By Month

In [7]:
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
abbr = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'july', 'aug', 'sept', 'aug', 'oct' ,'nov' ,'dec']
# data sets by month
print(0)
jan_data = xr.ufuncs.sqrt(wind_u.loc[wind_u['time.month'].isin([1])]**2 + wind_v.loc[wind_v['time.month'].isin([1])]**2)
print(1)
feb_data = xr.ufuncs.sqrt(wind_u.loc[wind_u['time.month'].isin([2])]**2 + wind_v.loc[wind_v['time.month'].isin([2])]**2)
print(2)
mar_data = xr.ufuncs.sqrt(wind_u.loc[wind_u['time.month'].isin([3])]**2 + wind_v.loc[wind_v['time.month'].isin([3])]**2)
print(3)
apr_data = xr.ufuncs.sqrt(wind_u.loc[wind_u['time.month'].isin([4])]**2 + wind_v.loc[wind_v['time.month'].isin([4])]**2)
print(4)
may_data = xr.ufuncs.sqrt(wind_u.loc[wind_u['time.month'].isin([5])]**2 + wind_v.loc[wind_v['time.month'].isin([5])]**2)
print(5)
jun_data = xr.ufuncs.sqrt(wind_u.loc[wind_u['time.month'].isin([6])]**2 + wind_v.loc[wind_v['time.month'].isin([6])]**2)
print(6)
july_data = xr.ufuncs.sqrt(wind_u.loc[wind_u['time.month'].isin([7])]**2 + wind_v.loc[wind_v['time.month'].isin([7])]**2)
print(7)
aug_data = xr.ufuncs.sqrt(wind_u.loc[wind_u['time.month'].isin([8])]**2 + wind_v.loc[wind_v['time.month'].isin([8])]**2)
print(8)
sept_data = xr.ufuncs.sqrt(wind_u.loc[wind_u['time.month'].isin([9])]**2 + wind_v.loc[wind_v['time.month'].isin([9])]**2)
print(9)
oct_data = xr.ufuncs.sqrt(wind_u.loc[wind_u['time.month'].isin([10])]**2 + wind_v.loc[wind_v['time.month'].isin([10])]**2)
print(10)
nov_data = xr.ufuncs.sqrt(wind_u.loc[wind_u['time.month'].isin([11])]**2 + wind_v.loc[wind_v['time.month'].isin([11])]**2)
print(11)
dec_data = xr.ufuncs.sqrt(wind_u.loc[wind_u['time.month'].isin([11])]**2 + wind_v.loc[wind_v['time.month'].isin([12])]**2)
print('done')

0
1
2
3
4
5
6
7
8
9
10
11
done


In [9]:
jan_data.to_netcdf(path='jan_data.netcdf', mode='w', format=None, group=None, engine=None, encoding=None, unlimited_dims=None, compute=True)
feb_data.to_netcdf(path='feb_data.netcdf', mode='w', format=None, group=None, engine=None, encoding=None, unlimited_dims=None, compute=True)
mar_data.to_netcdf(path='mar_data.netcdf', mode='w', format=None, group=None, engine=None, encoding=None, unlimited_dims=None, compute=True)
apr_data.to_netcdf(path='apr_data.netcdf', mode='w', format=None, group=None, engine=None, encoding=None, unlimited_dims=None, compute=True)
may_data.to_netcdf(path='may_data.netcdf', mode='w', format=None, group=None, engine=None, encoding=None, unlimited_dims=None, compute=True)
jun_data.to_netcdf(path='jun_data.netcdf', mode='w', format=None, group=None, engine=None, encoding=None, unlimited_dims=None, compute=True)
july_data.to_netcdf(path='july_data.netcdf', mode='w', format=None, group=None, engine=None, encoding=None, unlimited_dims=None, compute=True)
aug_data.to_netcdf(path='aug_data.netcdf', mode='w', format=None, group=None, engine=None, encoding=None, unlimited_dims=None, compute=True)
sept_data.to_netcdf(path='sept_data.netcdf', mode='w', format=None, group=None, engine=None, encoding=None, unlimited_dims=None, compute=True)
oct_data.to_netcdf(path='oct_data.netcdf', mode='w', format=None, group=None, engine=None, encoding=None, unlimited_dims=None, compute=True)
nov_data.to_netcdf(path='nov_data.netcdf', mode='w', format=None, group=None, engine=None, encoding=None, unlimited_dims=None, compute=True)
dec_data.to_netcdf(path='dec_data.netcdf', mode='w', format=None, group=None, engine=None, encoding=None, unlimited_dims=None, compute=True)